<a href="https://colab.research.google.com/github/ChihabEddine98/AmazonAnalysis/blob/main/AmazonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements Installations

In [3]:
# Run just once 
!pip install webdriver-manager
!pip install selenium
!pip install requests
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

zsh:1: command not found: apt-get
Unable to locate an executable at "/Users/djelidaymen/Library/Java/JavaVirtualMachines/openjdk-15/Contents/Home/bin/apt" (-1)
cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [12]:
# imports
import os
import sys
import requests as rq
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
import unicodedata
# end imports

ModuleNotFoundError: No module named 'requests'

# Part 01  : Data Gathering

In [13]:
# Settings 
reviews_div_cls = 'review'

prod_urls = {'informatique':[
    {
        'prod_url':
        'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/dp/B07X8NL7MW/ref=sr_1_7? __mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1616930888&  s=computers&sr=1-7',
    'rev_url': 'https://www.amazon.fr/Microsoft-Surface-Pro-Hybride-stockage/product-reviews/B07X8NL7MW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
    
    }
    ]
}

ds_cols = ['Rev_Title','Rev_Rate','Rev_Bdy','Rev_Hlp','Prod_Title','Prod_Brnd','Prod_Rate','Prod_Eval','Prod_Price','Category']


# End Settings

# Scrapping Product Information

In [17]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text


    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,       prod_rate,prod_eval,prod_price,prod_category]))
    return result



# Scrapping Reviews Data

In [16]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        try :
          review_title = review.find('a',{'data-hook' : 'review-title'}).text
        except AttributeError:
          review_title = 'Null'
        
        try :
          review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        except AttributeError:
          review_rate = '0'
        
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        
        try :
          review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        except AttributeError:
          review_help = '0'
        
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+prod)

    return ds

# Testing data collect

## Setup the web-driver 

In [15]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options = options)

NameError: name 'webdriver' is not defined

## Scrapping Products

In [45]:
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])
        print(product)

('Microsoft Surface Pro 7 Ordinateur Portable (Windows 10, écran tactile 12.3", Intel Core i5, 8Go RAM, 128Go SSD, Platine) PC Hybride polyvalent & performant', 'Marque : Microsoft Surface', '4,7 sur 5 étoiles', '3 274 évaluations', '991,98 €', 'informatique')


## Scrapping Reviews Data




In [53]:
ds = []
for category,products in prod_urls.items():
    for prod in products:
        for i in range(1,11):
            url = f'{prod["rev_url"]}&pageNumber={i}'
            ds = reviews_scrapper(driver,product,url,ds) 

df = pd.DataFrame(ds)
df.columns = ds_cols
df





,Rev_Title,Rev_Rate,Rev_Bdy,Rev_Hlp,Prod_Title,Prod_Brnd,Prod_Rate,Prod_Eval,Prod_Price,Category
0,Surface Pro 7ème génération,"4,0 sur 5 étoiles",Je possède la Surface Pro 6 (soit la généra...,155 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
1,Du reconditionné au prix du neuf!,"1,0 sur 5 étoiles",Produit bien reçu.Par contre on achète au pr...,110 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
2,THROTTING,"2,0 sur 5 étoiles",POUR LA VERSION I5 SEULEMENT : PHENOMENE DE TH...,72 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
3,ATTENTION AU TITRE TROMPEUR !,"2,0 sur 5 étoiles",Le titre fait référence à un Surface Pro 7 ...,98 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
4,C'est bien une version i7,"5,0 sur 5 étoiles",Contrairement au commentaire disant que c'est ...,44 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
...,...,...,...,...,...,...,...,...,...,...
95,,0,Mein Freund hat sich das Surface Pro 7 bestell...,120 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
96,,0,Glaubt nicht den Rezessenten die behaupten die...,110 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
97,,0,"Un ottimo pc, che vuole essere un tablet, con ...",90 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
98,,0,"Un tablet che non sa fare il tablet, un ultrab...",40 personnes ont trouvé cela utile,Microsoft Surface Pro 7 Ordinateur Portable (W...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"991,98 €",informatique
